In [5]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
from io import StringIO
from shapely.geometry import Point
# Use your own path here
os.chdir("/Users/canyonfoot/Documents/python_proj/EPA-Hack-Day-Analysis")

from typing import Optional

import pickle
import numpy as np

pd.options.mode.chained_assignment = None

In [24]:
RMP_data = gpd.read_file("data/processed/facilities_geo.geojson")
naics_mapping = gpd.read_file("data/processed/epa_naics_mapping.csv").reset_index()
gdf_urban_all = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER_RD18/LAYER/UAC20/tl_rd22_us_uac20.zip").rename(columns = {"NAME20": "UA_NAME"})[['UA_NAME','geometry']]
UA_stats = pd.read_csv("data/processed/urban_area_statistics.csv").query("facility_count >= 10")

In [25]:
gdf_urban_all = gdf_urban_all.merge(UA_stats["UA_NAME"])

In [27]:
RMP_data.NAICSCode = RMP_data.NAICSCode.astype(str)
RMP_data.NAICS_3 = RMP_data.NAICS_3.astype(str)

In [29]:
conditions = [
    (RMP_data['NAICS_3'] == "325"),
    (RMP_data['NAICS_3'] == "331"),
    (RMP_data['NAICS_3'] == "311"),
    (RMP_data['NAICSCode'].str.slice(0,4) == "2122"),
    (RMP_data['NAICS_3'] == "324"),
    (RMP_data['NAICS_3'] == "322"),
    (RMP_data['NAICSCode'].str.slice(0,4) == "2211"),
    (RMP_data['NAICSCode'] == "562112") | (RMP_data['NAICSCode'] == "562112")
]

choices = ["Chemical Manufacturing",
"Primary Metals",
"Food Manufacturing",
"Metal Mining",
"Petroleum Products Manufacturing",
"Paper Manufacturing",
"Electric Utilities",
"Hazardous Waste Management"]

In [30]:
RMP_data["EPA_industry"] = np.select(conditions, choices, "Other")

In [31]:
RMP_UA = RMP_data.overlay(gdf_urban_all.to_crs(4326))

In [35]:
RMP_UA["Description"] = "Urban Area: " + RMP_UA["UA_NAME"] + '/n' + "Facility Industry: " + RMP_data["EPA_industry"]

In [39]:
gdf_urban_all["geometry"] = gdf_urban_all.simplify(0.001)

In [55]:
gdf_urban_all[["UA_NAME", "geometry"]].to_file("viz/data/urban_areas.geojson")
RMP_UA[["EPAFacilityID", "Description", "geometry"]].to_file("viz/data/RMP_facilities.geojson")